In [17]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import imghdr
import tensorflow as tf
from PIL import Image
import seaborn as sns

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.applications import VGG16, ResNet50V2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import plot_model
from sklearn.metrics import f1_score

In [18]:
train_data_dir = '/Users/abhishekwaghchaure/Desktop/PWSkillsDataScience/DataSets/Fer2013/train'
test_data_dir = '/Users/abhishekwaghchaure/Desktop/PWSkillsDataScience/DataSets/Fer2013/test'

In [21]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.metrics import Precision, Recall

# Step 1: Set the image size for MobileNet and number of classes
img_width, img_height = 224, 224  
input_shape = (img_width, img_height, 3)  
classes = 6


base_model = MobileNet(input_shape=input_shape, include_top=False, weights='imagenet')

# Step 3: Freeze the base model layers (optional, can fine-tune later)
base_model.trainable = False

# Step 4: Add custom classification layers on top of MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu')(x) 
x = Dropout(0.5)(x) 
predictions = Dense(classes, activation='softmax')(x) 

# Step 5: Define the full model
model = Model(inputs=base_model.input, outputs=predictions)

precision = Precision()
recall = Recall()

# Step 6: Compile the model with a suitable optimizer, loss, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', precision, recall])

# Step 7: Print model summary
model.summary()

# Step 8: Prepare the data (train, validation, test)
# Using ImageDataGenerator for data augmentation and rescaling images
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255,  # Rescale the pixel values from [0, 255] to [0, 1]
    rotation_range=40,  # Degree range for random rotations
    width_shift_range=0.2,  # Range (as a fraction of total width) for random horizontal shifts
    height_shift_range=0.2,  # Range (as a fraction of total height) for random vertical shifts
    shear_range=0.2,  # Shearing intensity (shear angle in counter-clockwise direction)
    zoom_range=0.2,  # Range for random zoom
    horizontal_flip=True,  # Randomly flip inputs horizontally
    fill_mode='nearest',  # Strategy to fill newly created pixels, which can appear after a rotation or a width/height shift
    validation_split=0.2 # Randomly flip images horizontally
)

# test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for test/validation data
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # Only MobileNet-specific preprocessing
)

# # Step 9: Load the training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Path to your training data
    target_size=(img_width, img_height),  # Resize all images to 224x224
    batch_size=64,  # Batch size
    class_mode='categorical',  # Categorical labels (one-hot encoding)
    subset='training',
    shuffle = True
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Path to your validation data
    target_size=(img_width, img_height),  # Resize to 224x224
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    shuffle = False
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,  # Path to your test data
    target_size=(img_width, img_height),  # Resize to 224x224
    batch_size=64,
    class_mode='categorical'
)

# Check one batch from the generator
x_batch, y_batch = next(train_generator)
print(f"x_batch shape: {x_batch.shape}")
print(f"y_batch shape: {y_batch.shape}")

# train_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,  # MobileNet-specific preprocessing
#     rotation_range=30,  # Random rotations for augmentation
#     width_shift_range=0.2,  # Horizontal shift for augmentation
#     height_shift_range=0.2,  # Vertical shift for augmentation
#     shear_range=0.2,  # Shearing for augmentation
#     zoom_range=0.2,  # Random zoom for augmentation
#     horizontal_flip=True,  # Horizontal flip for augmentation
#     fill_mode='nearest'  # Filling in missing pixels
# )

# # Validation/test data generator without augmentation, just preprocessing for MobileNet
# test_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input  # Only MobileNet-specific preprocessing
# )

# # Train generator
# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,  # Directory containing training data
#     target_size=(img_width, img_height),# Resize images to 224x224
#     batch_size=32,  # Set batch size based on your memory capacity
#     class_mode='categorical',  # For multi-class classification
#     shuffle=True  # Shuffle training data
# )

# # Validation generator
# validation_generator = test_datagen.flow_from_directory(
#     test_data_dir,  # Directory containing validation data
#     target_size=(img_width, img_height),# Resize images to 224x224
#     batch_size=32,  # Set batch size based on your memory capacity
#     class_mode='categorical',  # For multi-class classification
#     shuffle=False  # No shuffling for validation data
# )

#     # Test generator
# test_generator = test_datagen.flow_from_directory(
#     test_data_dir,  # Path to your test data
#     target_size=(img_width, img_height),  # Resize to 224x224
#     batch_size=32,
#     class_mode='categorical',
#     shuffle=False  # No shuffling for validation data
# )

# Step 10: Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 64,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 64,
    epochs=60
)

# Step 11: Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_acc:.4f}') 
print(f'Test accuracy: {test_acc:.4f}') 
print(f'Test accuracy: {test_acc:.4f}')

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 4,284,614 (16.34 MB)

 Trainable params: 1,055,750 (4.03 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

...

2024-10-13 10:25:16.596463: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-13 10:25:16.596481: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12350376671583507878
2024-10-13 10:25:16.596488: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 15243586415784565779
2024-10-13 10:25:16.596509: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


353/353 ━━━━━━━━━━━━━━━━━━━━ 205s 571ms/step - accuracy: 0.2573 - loss: 1.7613 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_accuracy: 0.2562 - val_loss: 1.7550 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00
Epoch 4/60
  1/353 ━━━━━━━━━━━━━━━━━━━━ 1:30 258ms/step - accuracy: 0.2188 - loss: 1.7587 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00

2024-10-13 10:28:41.930517: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-13 10:28:41.930545: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6859170967929151648
2024-10-13 10:28:41.930550: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 1274187057771946310
2024-10-13 10:28:41.930582: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_18]]
2024-10-13 10:28:41.930591: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14768735908985700540
2024-10-13 10:28:41.930599: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11901049637740560423
2024-10-

353/353 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step - accuracy: 0.2188 - loss: 1.7587 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 2.2110 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00
Epoch 5/60
 25/353 ━━━━━━━━━━━━━━━━━━━━ 2:34 472ms/step - accuracy: 0.2356 - loss: 1.7602 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_curves(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    # Add checks for precision and recall in the history (they may not always be available)
    precision = history.history.get("precision", [])
    recall = history.history.get("recall", [])

    epochs = range(len(loss))

    plt.figure(figsize=(15, 5))

    # Plot loss
    plt.subplot(1, 3, 1)
    plt.plot(epochs, loss, label="Training Loss")
    plt.plot(epochs, val_loss, label="Validation Loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 3, 2)
    plt.plot(epochs, accuracy, label="Training Accuracy")
    plt.plot(epochs, val_accuracy, label="Validation Accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    # Plot precision and recall if available
    if precision and recall:
        plt.subplot(1, 3, 3)
        plt.plot(epochs, precision, label="Precision")
        plt.plot(epochs, recall, label="Recall")
        plt.title("Precision and Recall")
        plt.xlabel("Epochs")
        plt.ylabel("Value")
        plt.legend()

    plt.tight_layout()
    plt.show()

# Call the plot_curves function after training
plot_curves(history)


In [ ]:
MobileNet_Predictions = model.predict(test_generator)

# Choosing highest probalbilty class in every prediction 
MobileNet_Predictions = np.argmax(MobileNet_Predictions, axis=1)

In [ ]:
history = history
final_accuracy = history.history['accuracy'][-1]
final_precision = history.history['precision'][-1]
final_recall = history.history['recall'][-1]
final_val_loss = history.history['val_loss'][-1]
final_f1_score = (2 * final_precision * final_recall) / (final_precision + final_recall)

print("*" * 8, "Final Metrics", "*" * 8)
print("Final Accuracy        : {:.2%}".format(final_accuracy))
print("Final Precision       : {:.2%}".format(final_precision))
print("Final Recall          : {:.2%}".format(final_recall))
print("Final Validation Loss : {:.4f}".format(final_val_loss))
print("Final F1 Score        : {:.3}".format(final_f1_score))

In [ ]:
import seaborn as sns 
from sklearn.metrics import confusion_matrix

fig, ax= plt.subplots(figsize=(15,10))

cm=confusion_matrix(test_generator.labels, MobileNet_Predictions)

sns.heatmap(cm, annot=True, fmt='g', ax=ax)

ax.set_xlabel('Predicted labels', fontsize=15, fontweight='bold')
ax.set_ylabel('True labels', fontsize=15, fontweight='bold')
ax.set_title('MobileNet Confusion Matrix', fontsize=20, fontweight='bold')


In [ ]:
class_indices = test_generator.class_indices
Emotion_Classes = [key.capitalize() for key in class_indices.keys()]
print(Emotion_Classes)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

# Assuming 'test_generator' contains your test image data and 'Emotion_Classes' is a list of your class labels

def plot_predictions(model, test_generator, Emotion_Classes, num_images=10):
    # Randomly select a batch and images within the batch
    Random_batch = np.random.randint(0, len(test_generator) - 1)
    Random_Img_Index = np.random.randint(0, test_generator.batch_size - 1, num_images)

    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(25, 15), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        # Get random image and label from the selected batch
        Random_Img = test_generator[Random_batch][0][Random_Img_Index[i]]
        Random_Img_resized = resize(Random_Img, (224, 224), anti_aliasing=True)  # Resize for MobileNet
        
        # Get the true label and model prediction
        Random_Img_Label = np.argmax(test_generator[Random_batch][1][Random_Img_Index[i]])  # True label
        Model_Prediction = np.argmax(model.predict(Random_Img_resized.reshape(1, 224, 224, 3), verbose=0))  # Prediction

        # Display the image
        ax.imshow(Random_Img_resized)

        # Set title color based on correct/incorrect prediction
        if Emotion_Classes[Random_Img_Label] == Emotion_Classes[Model_Prediction]:
            color = "green"  # Correct prediction
        else:
            color = "red"  # Incorrect prediction

        # Set the title to show true vs predicted labels
        ax.set_title(f"True: {Emotion_Classes[Random_Img_Label]}\nPredicted: {Emotion_Classes[Model_Prediction]}", color=color, fontsize=18)

    plt.tight_layout()
    plt.show()

# Call the function to plot predictions
plot_predictions(model, test_generator, Emotion_Classes)